In [16]:
import sys
import os
from pathlib import Path

if (dir := str(Path(os.getcwd()).parent)) not in sys.path:
    sys.path.insert(0, dir)

import pandas as pd
from algorithm.evolution import evolution
import time
import numpy as np
from itertools import product

## Конфигурация системы

|  |  |
| ------ | ------ |
| Процессор | AMD Ryzen 5 5500U 2.10 GHz |
| ОЗУ | 16 ГБ |
| Операционная система | Windows 11 |


In [17]:
parameters = {
    'N': 8,
    "population_dim": 250,
    "tournament_size": 5,
    "crossing_probability": 0.85,
    "mutation_probability": 0.15,
    "exchange_num": 3,
    "steps": 200,
}

In [23]:
def grid_search(params: dict):
    """
    yield grid of params
    """
    param_grid = [product([k], np.unique(v)) for k, v in params.items()]
    for grid in product(*param_grid):
        yield dict(tuple(grid))

In [24]:
def evolution_experiment(evo_params: dict) -> dict:
    start_time = time.time()
    res = evolution(**evo_params)
    process_time = time.time() - start_time
    return {
        'Time': process_time,
        'Fitness': res['fitness'],
        'Cache Size': len(res['cache']),
        **evo_params
    }

In [28]:
def create_df(params: dict, times: int) -> pd.DataFrame:
    data = []
    for _ in range(times):
        if _ % 10 == 0:
            print(f'{_ + 1}/{times}')
        data.append(evolution_experiment(params))
    columns=['Time', 'Fitness', 'Cache Size', *parameters.keys()]
    return pd.DataFrame(data=data, columns=columns)

In [57]:
# загрузить с файла
# df = pd.read_csv('experiment_params.csv', sep=';')

# начать с нуля
df = create_df({}, 0)

df

,Time,Fitness,Cache Size,N,population_dim,tournament_size,crossing_probability,mutation_probability,exchange_num,steps


In [59]:
grid = {
    'N': [12],
    "population_dim": range(100, 301, 50),
    "tournament_size": [3],
    "crossing_probability": [0.85],
    "mutation_probability": [0.05],
    "exchange_num": [2],
    "steps": 300,
}

n_instance = 100

# new_df = [create_df(params, n_instance) for params in grid_search(grid)]
# df = pd.concat([df, *new_df], ignore_index=True)

In [60]:
for i, params in enumerate(grid_search(grid)):
    # if i < 37:
    #     continue
    print(f'{i+1:^9}')
    new_df = create_df(params, n_instance)
    df = pd.concat([df, new_df], ignore_index=True)

    1    
1/100
11/100
21/100
31/100
41/100
51/100
61/100
71/100
81/100
91/100
    2    
1/100
11/100
21/100
31/100
41/100
51/100
61/100
71/100
81/100
91/100
    3    
1/100
11/100
21/100
31/100
41/100
51/100
61/100
71/100
81/100
91/100
    4    
1/100
11/100
21/100
31/100
41/100
51/100
61/100
71/100
81/100
91/100
    5    
1/100
11/100
21/100
31/100
41/100
51/100
61/100
71/100
81/100
91/100


In [61]:
df

,Time,Fitness,Cache Size,N,population_dim,tournament_size,crossing_probability,mutation_probability,exchange_num,steps
0,10.899293,4,300,12,100,3,0.85,0.05,2,300
1,10.491676,2,300,12,100,3,0.85,0.05,2,300
2,10.535703,8,300,12,100,3,0.85,0.05,2,300
3,10.436162,4,300,12,100,3,0.85,0.05,2,300
4,10.688807,5,300,12,100,3,0.85,0.05,2,300
...,...,...,...,...,...,...,...,...,...,...
495,32.573833,5,300,12,300,3,0.85,0.05,2,300
496,33.007051,4,300,12,300,3,0.85,0.05,2,300
497,32.010085,5,300,12,300,3,0.85,0.05,2,300
498,31.536219,4,300,12,300,3,0.85,0.05,2,300


In [62]:
df.groupby(['population_dim']).mean()

,Time,Fitness,Cache Size,N,tournament_size,crossing_probability,mutation_probability,exchange_num,steps
population_dim,,,,,,,,,
100,10.467490,4.59,292.75,12.0,3.0,0.85,0.05,2.0,300.0
150,15.351652,4.41,291.96,12.0,3.0,0.85,0.05,2.0,300.0
200,19.476842,3.77,276.72,12.0,3.0,0.85,0.05,2.0,300.0
250,25.721902,3.85,286.57,12.0,3.0,0.85,0.05,2.0,300.0
300,29.052803,3.47,270.76,12.0,3.0,0.85,0.05,2.0,300.0


In [55]:
df.groupby(['N', 'population_dim','tournament_size', 'crossing_probability', 'mutation_probability', 'exchange_num']).mean()

Time   
N population_dim tournament_size crossing_probability mutation_probability exchange_num             
8 100            3               0.65                 0.05                 1             2.236117  \
                                                                           2             1.165862   
                                                                           3             1.197657   
                                                      0.15                 1             0.834105   
                                                                           2             0.467957   
                                                                           3             0.384357   
                                                      0.25                 1             0.458174   
                                                                           2             0.291316   
                                                                           3             0.233072   
                                 0.75                 0.05                 1             2.328493   
                                                                           2             1.343231   
                                                                           3             1.125260   
                                                      0.15                 1             0.993486   
                                                                           2             0.475491   
                                                                           3             0.421732   
                                                      0.25                 1             0.447455   
                                                                           2             0.293471   
                                                                           3             0.209608   
                                 0.85                 0.05                 1             2.387395   
                                                                           2             1.482766   
                                                                           3             1.197772   
                                                      0.15                 1             0.928189   
                                                                           2             0.530723   
                                                                           3             0.378001   
                                                      0.25                 1             0.535742   
                                                                           2             0.225970   
                                                                           3             0.254421   
                                 0.95                 0.05                 1             2.532249   
                                                                           2             1.499379   
                                                                           3             1.031290   
                                                      0.15                 1             0.988292   
                                                                           2             0.532573   
                                                                           3             0.334248   
                                                      0.25                 1             0.371477   
                                                                           2             0.225836   
                                                                           3             0.250671   
                 5               0.65                 0.05                 1             1.559363   

                                                                                        Fitness   
N population_dim tournament_size crossing_probability 

In [56]:
df.to_csv('experiment_population_dim.csv', index=False, sep=';')

### Изменение датасета

In [ ]:
z = create_df(0, 0)
z

In [ ]:
df_to_change = df[(df['N'] == ...) & (df['fitness'] > ...)]
df_to_change

In [ ]:
idx = df_to_change.index
idx

In [ ]:
df.loc[idx]

In [ ]:
z.index = idx
z

In [162]:
df.update(z, 'left')